In [ ]:
import warnings; warnings.simplefilter('ignore')
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
oe = preprocessing.OneHotEncoder(sparse=False)
import dlib
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import timeit
label_encoder = preprocessing.LabelEncoder()
import pickle
import random

# Task 3: Structured SVM 

In [ ]:
# Windows Length
L = 2
# Number of examples
N = 5000
# Length of a feature
d = 128
# The hyper-parameter for icm search
Niter = 2               

In [ ]:
def l2i(a):
        return int(ord(a)-ord('a'))
def i2l(i):
    if i >= 0:
        return chr(i+ord('a'))
    else:
        return '_'
def iors(s):
    try:
        return int(s)
    except ValueError: # if it is a string, return a string
        return s

In [ ]:
# Read the entire dataset into lists or list of lists
def read_OCR(filename, n_features):
    F = open(filename)
    dataset = {}
    dataset['ids'] = []#np.zeros(n_examples, dtype=int)
    dataset['labels'] = []#np.zeros(n_examples,dtype=int)
    dataset['labelDic'] = {} # To profile the distribution of labels
    dataset['next_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['word_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['positions'] = []#np.zeros(n_examples,dtype=int)
    dataset['folds'] = []#np.zeros(n_examples,dtype=int)
    dataset['features'] = []#np.zeros([n_examples,n_features])
    
    for str_line in F.readlines():
        #line0 = map(iors, filter(None, re.split('\t', str_line.strip())))
        ## ATTENTION: If you are using Python3, use the following line instead
        line0 = list(map(iors, filter(None, re.split('\t', str_line.strip()))))


        dataset['ids'].append(int(line0.pop(0)))
        dataset['labels'].append(l2i(line0.pop(0))) # The label is converted into integer('a'=>0, 'z'=>25)
        if dataset['labels'][-1] in dataset['labelDic']:
            dataset['labelDic'][dataset['labels'][-1]] += 1
        else:
            dataset['labelDic'][dataset['labels'][-1]] = 1
            
        dataset['next_ids'].append(int(line0.pop(0)))
        dataset['word_ids'].append(int(line0.pop(0)))
        dataset['positions'].append(int(line0.pop(0)))
        dataset['folds'].append(int(line0.pop(0)))
        if len(line0) != 128:  # Sanity check of the length
            print (len(line0))
        dataset['features'].append(line0)

    return dataset

In [ ]:
dataset1 = read_OCR('OCRdataset/letter.data', d)

In [ ]:
# Understand the profile of OCR raw data
print ("max of labels=", max(dataset1['labels']), " min of labels=", min(dataset1['labels']), 'num of labels=', len(dataset1['labelDic']))
print ("labelDic.keys()=", map(i2l, dataset1['labelDic'].keys()))
print ("Total number of lines=", len(dataset1['ids']))
print ("The shape of features:", np.array(dataset1['features']).shape)

print ("The first 10 ids:",dataset1['ids'][:10])
print ("ids[0]=",dataset1['ids'][0])
print ("labels[0]=", dataset1['labels'][0])
print ("The 1st letter is ", i2l(dataset1['labels'][0]))
print ("next_ids[0]=",dataset1['next_ids'][0])
# Show the matrix into an image
def showFeatures(features, num):
    plt.figure(figsize=(num, 6))
    
    for i in range(num):
        npfeature = np.array(features[i])
        plt.subplot(1,num,i+1)
        imshow(npfeature.reshape(16,8), cmap='gray')
        plt.title(i)

showFeatures(dataset1['features'],20)

# f1 = np.array(dataset1['features'])
# f1 = np.hstack([np.ones((f1.shape[0],1)), f1])
# print f1.shape
# dataset1['features'] = f1.tolist()
# d +=1

# Options for Chopping

## Concatenating and structurizing
### version one:
####  (1) find 5000 words, split them into 4000 for training, and 1000 for testing. 
####  (2) chop first 2 characters from each word
####  (3) then, you will have 4000 two-words pairs for training, and 1000 for testing. 
####  (4) construct new structures data based on this pairs. 
 
  e.g.
  
  apple => ap
  
  banana => ba
  
 
### version two:
####  (1) continue reading words from data until you have 4001 characters for training, and 1001 for testing
####  (2) concatenate characters together to form structure features. 
####  (3) you will have the same amount structured data. 
  apple => ap, pp, pl, le
 
 banana => ba, an, na, an, na
  
### version three
####  (1) similar to version two, but add a dummy letter '_' between words.
  apple_banana => ap, pp, pl, le, e_, _b, ba, an, na, an, na
 
### version four
#### (1) similar, but window stride is 2 also, and pad a dummy letter '_', which will be represented as all 0 in the image. 
  apple => ap, pl, e_
  
  banana => ba, na,  na

In [ ]:
# Option 1: Extract the first L letters in a word
# You are welcome to try other options

def structurize1(dataset, N, L):
    d_features = len(dataset['features'][0])
    y = dataset['labels']
    X = dataset['features']
    next_id = dataset['next_ids']

    labels = np.zeros((N, L))
    features = np.zeros((N, L*d_features))
    
    # Extract only one structured example
    def extract(iN, loc):
        labels[iN] = y[loc:loc+L]
        features[iN] = np.array(X[loc:loc+L]).ravel().tolist()
        iN += 1
        return iN
    
    iN = 0
    iN = extract(iN, 0)
    
    for key, value in enumerate(y):
        if next_id[key] == -1:
            iN = extract(iN, key+1)
            
            if iN == N:
                break
    
    c = list(zip(labels, features))
    random.shuffle(c)
    labels, features = zip(*c)
    
    return np.array(labels), np.array(features)      

In [ ]:
labels1, features1 = structurize1(dataset1, N, L)

print (np.all(labels1[labels1==labels1[0]]))
print (labels1[:15].T)
showFeatures(features1[:,0:128],15)
showFeatures(features1[:,128:256],15)

# Use dlib to classify

In [ ]:
print ('N=',N,'L=',L,'d=',d)

In [ ]:
class ThreeClassProblem:
    C = 1

    def __init__(self, samples, labels, L, K, d):
        self.L = L
        self.K = K
        self.d = d
        self.num_samples = len(samples)
        self.num_dimensions = (L*K*d+1) + (L-1)
        self.samples = samples
        self.labels = labels
        self.loss_for_loop = True
        
    def make_psi(self, x, label):
       
        psi = dlib.vector()
        psi.resize(self.num_dimensions)
        psi[0] = x[0] # The bias
        
        ########### YOUR CODE HERE ###########
        
        return psi

    def get_truth_joint_feature_vector(self, idx):
        return self.make_psi(self.samples[idx], self.labels[idx])
    
    def separation_oracle(self, idx, current_solution):
        samp = self.samples[idx]
        psi = [0]*self.num_dimensions
        max1 = -1e10
        max_scoring_label = [0]*L # Initialize max_scoring_label for icm search
        for k in range(Niter):
            for iL in range(self.L):   # Iterate over the window length
                for i in range(self.K):# Change different label for the search of a structured label
                    tmp_label = ########### YOUR CODE HERE ########### # New a list to avoid modifying the max_scoring_label
                    tmp_label[iL] = ########### YOUR CODE HERE ########### # Take turns to modify the structured label from left to right. The guessed structured label.
                    tmp_psi = ########### YOUR CODE HERE ########### # Make a new Psi for the guessed structured label
                    score1 = dlib.dot(current_solution, tmp_psi)
                    
                    loss1 = 0.0
                    if self.loss_for_loop:
                        for j in range(self.L):
                            if self.labels[idx][j] != tmp_label[j]:
                                loss1 += 1.0
                    else:
                        if self.labels[idx] != tmp_label: # Add the conditional "1"
                            loss1 += 1.0

                    if max1 < score1+loss1: # Search for the maximum and update loss, max_scoring_label, and psi
                        max1 = score1 + loss1
                        loss = loss1
                        max_scoring_label[iL] = i
                        psi = tmp_psi

        return loss, psi

In [ ]:
def cal_accuracy(samples, labels, problem, weights, K):
    predictions = []
    for samp in samples:
        prediction = [0]*L # Initialize max_scoring_label for icm search
        Niter = 2                 # The hyper-parameter for icm search
        max1 = -1e10              # The max value during maximizing our target function
        for k in range(Niter):
            for iL in range(L):   # Iterate over the window length
                for i in range(K):# Change differnet label for the search of a structured label
                    tmp_label = ########### YOUR CODE HERE ###########
                    tmp_label[iL] = ########### YOUR CODE HERE ###########
                    psi1 = ########### YOUR CODE HERE ###########
                    score1 = dlib.dot(weights, psi1)

                    if max1 < score1:
                        max1 = score1
                        prediction[iL] = i
        predictions.append(prediction)
    
    
    errCnt = 0
    for i in range(len(predictions)):
        if predictions[i] != labels[i]:
            errCnt += 1

    return 1.0-float(errCnt)/float(len(predictions))

In [ ]:
le1 = preprocessing.LabelEncoder()
nplabels1  = le1.fit_transform(labels1.ravel()).reshape(labels1.shape)
npsamples1 = np.hstack([np.ones((N,1)), features1]) # Add ones for bias
K1 = len(le1.classes_)
print ('K1=', K1)

tr_labels  = nplabels1[:int(N*0.8)].astype(int).tolist()
tr_samples = npsamples1[:int(N*0.8)].astype(int).tolist()
te_labels  = nplabels1[int(N*0.8):].astype(int).tolist()
te_samples = npsamples1[int(N*0.8):].astype(int).tolist()

def profiling(labels):
    TrDic = {}
    for i in np.array(labels).ravel():
        if i not in TrDic:
            TrDic[i] = 1
        else:
            TrDic[i] += 1
    return TrDic
print (profiling(tr_labels))
print (profiling(te_labels))

In [ ]:
problem = ThreeClassProblem(tr_samples, tr_labels, L, K1, d)

start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open('weights1_1.obj', 'wb'))
#pickle.dump(weights, open('weights1_1.obj', 'w'))

In [ ]:
weights_load = pickle.load(open('weights1_1.obj','rb'))
#weights_load = pickle.load(open('weights1_1.obj','r'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K1))
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K1))